### Find expired registrations and visualize data on an interactive map

**About:**


- I am leveraging US Addresses POI data from the marketplace and joining it with data related to my DMV use case.  
- This demo has been designed so you can join location data to data that may be specific to your use case, but does not contain any address or geo coded data.  
- Customize the data to the desired location by filtering the POI data.
- There are five Streamlit maps that incorporate layers - H3HexagonLayer, ScatterplotLayer, ArcLayer, PathLayer.  Included in the maps are tooltips and filters to select a location of interest and points within a specified distance.
- **Use this as is or use the working examples to build your own custom demo….**



**Concepts:**
- Run SQL and python in notebook
- Import python packages
- Create DB and Schema
- Create tables in RAW layer
- Access Snowflake Marketplace for address data
- Join tables to create SILVER layer
- Create interactive map

### Add the following package via the Packages dropdown:


- pydeck

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import pydeck as pdk

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
get_active_session()

In [ ]:
--drop database DEMO_GEOSPATIAL;
create or replace database DEMO_GEOSPATIAL;
create schema raw;
create schema silver;
create schema gold;

In [ ]:
use database DEMO_GEOSPATIAL;
use role accountadmin;
 


In [ ]:
create or replace TABLE RAW.USECASE_TABLE (
	REGISTRATION_CLASS VARCHAR(16777216),
	VIN VARCHAR(16777216),
	MODEL_YEAR VARCHAR(16777216),
	MAKE_CODE VARCHAR(16777216),
	BODY_TYPE VARCHAR(16777216),
	MAXIMUM_GROSS_VEHICLE_WEIGHT NUMBER(38,0),
	SEATS VARCHAR(16777216),
	FUEL VARCHAR(16777216),
	CYLINDERS VARCHAR(16777216),
	TRANSACTION VARCHAR(16777216),
	REGISTRANT VARCHAR(16777216),
	CITY VARCHAR(16777216),
	STATE VARCHAR(16777216),
	ZIP_CODE VARCHAR(16777216),
	RESIDENCE_COUNTY VARCHAR(16777216),
	EFFECTIVE_DATE DATE,
	REGISTRATION_EXPIRATION DATE,
	LENGTH VARCHAR(16777216),
	HULL_MATERIAL VARCHAR(16777216),
	PROPULSION VARCHAR(16777216),
	USE VARCHAR(16777216),
	COUNTY_OF_USE VARCHAR(16777216),
	DISPLACEMENT_OR_HORSEPOWER VARCHAR(16777216),
	EXPORTED DATE,
	GEOREFERENCE GEOGRAPHY
);


In [ ]:
-- This cell populates enough sample data so the maps display
--
--To load a more full dataset see below:
-- Downlod csv from: 
--     https://catalog.data.gov/dataset/department-of-motor-vehicles-registration-transactions-two-year-window
--     it's a huge file.
--     I created a file of 100k records via the terminal command:
--  cat Department_of_Motor_Vehicles_Registration_Transactions__Two_Year_Window.csv | head -100000 > highway_safety_100000.csv
--    I then loaded into RAW.USECASE_TABLE

insert into RAW.USECASE_TABLE
(   VIN,  REGISTRATION_EXPIRATION  )
values
('KNDJ23AU6N7161727','12/01/2021'),
('KNDJ23AU6N7161728','12/21/2023'),
('KNDJ23AU6N7141727','12/01/2023'),
('KNDJ23AU6N716728','11/21/2024'),
('KNDJ23AU6N7131727','1/01/2024'),
('KNDJ23AU6N7141728','12/21/2022'),
('KNDJ23AU6N7151727','12/01/2022'),
('KNDJ23AU6N7161728','12/21/2022'),
('KNDJ23AU6N7171727','12/01/2022');

-- blow up your sample data to 4,608 records

insert into RAW.USECASE_TABLE( VIN,  REGISTRATION_EXPIRATION) select VIN,  REGISTRATION_EXPIRATION from RAW.USECASE_TABLE ;
insert into RAW.USECASE_TABLE( VIN,  REGISTRATION_EXPIRATION) select VIN,  REGISTRATION_EXPIRATION from RAW.USECASE_TABLE ;
insert into RAW.USECASE_TABLE( VIN,  REGISTRATION_EXPIRATION) select VIN,  REGISTRATION_EXPIRATION from RAW.USECASE_TABLE ;
insert into RAW.USECASE_TABLE( VIN,  REGISTRATION_EXPIRATION) select VIN,  REGISTRATION_EXPIRATION from RAW.USECASE_TABLE ;
insert into RAW.USECASE_TABLE( VIN,  REGISTRATION_EXPIRATION) select VIN,  REGISTRATION_EXPIRATION from RAW.USECASE_TABLE ;
insert into RAW.USECASE_TABLE( VIN,  REGISTRATION_EXPIRATION) select VIN,  REGISTRATION_EXPIRATION from RAW.USECASE_TABLE ;
insert into RAW.USECASE_TABLE( VIN,  REGISTRATION_EXPIRATION) select VIN,  REGISTRATION_EXPIRATION from RAW.USECASE_TABLE ;
insert into RAW.USECASE_TABLE( VIN,  REGISTRATION_EXPIRATION) select VIN,  REGISTRATION_EXPIRATION from RAW.USECASE_TABLE ;
insert into RAW.USECASE_TABLE( VIN,  REGISTRATION_EXPIRATION) select VIN,  REGISTRATION_EXPIRATION from RAW.USECASE_TABLE ;
select count(*) from  RAW.USECASE_TABLE; 


In [ ]:
-- Step below do the following:
-- 1) create an empty table based on existing table
-- 2) add a sequence to the table
-- 3) populate the table with a select into
--
-- The result is table with a row_id containing unique values
-- The premise is this table has no address or geo data
-- The table can contain any data to support the story you want to tell
--  Location info will be incporated in following cells
--
-- If you want to reproduce this exact demo see the cell below for the HIGHWAY_SAFETY ddl and data
--
create or replace table RAW.HIGHWAY_SAFETY as 
select 
REGISTRATION_CLASS, VIN, MODEL_YEAR, MAKE_CODE, BODY_TYPE, MAXIMUM_GROSS_VEHICLE_WEIGHT, SEATS, FUEL, CYLINDERS, TRANSACTION, REGISTRANT, EFFECTIVE_DATE, REGISTRATION_EXPIRATION, LENGTH, HULL_MATERIAL, PROPULSION, USE, DISPLACEMENT_OR_HORSEPOWER, EXPORTED,
from 
RAW.USECASE_TABLE limit 0;
--------------------------------------------------------------------------------

-- Create sequence and add it to the table, as data gets added the ROW_ID will have unique values
-- the ROW_ID will be used to join to the address table
--

CREATE OR REPLACE SEQUENCE RAW.HIGHWAY_SAFETY_ROW_ID_SEQ
  START WITH 1
  INCREMENT BY 1;

alter table RAW.HIGHWAY_SAFETY add column
ROW_ID NUMBER(38,0) DEFAULT RAW.HIGHWAY_SAFETY_ROW_ID_SEQ.NEXTVAL;
--------------------------------------------------------------------------------  

-- add data to new table and verify the sequency is working
--
insert into RAW.HIGHWAY_SAFETY
( REGISTRATION_CLASS, VIN, MODEL_YEAR, MAKE_CODE, BODY_TYPE, MAXIMUM_GROSS_VEHICLE_WEIGHT, SEATS, FUEL, CYLINDERS, TRANSACTION, REGISTRANT, EFFECTIVE_DATE, REGISTRATION_EXPIRATION, LENGTH, HULL_MATERIAL, PROPULSION, USE, DISPLACEMENT_OR_HORSEPOWER, EXPORTED)
select 
REGISTRATION_CLASS, VIN, MODEL_YEAR, MAKE_CODE, BODY_TYPE, MAXIMUM_GROSS_VEHICLE_WEIGHT, SEATS, FUEL, CYLINDERS, TRANSACTION, REGISTRANT, EFFECTIVE_DATE, REGISTRATION_EXPIRATION, LENGTH, HULL_MATERIAL, PROPULSION, USE, DISPLACEMENT_OR_HORSEPOWER, EXPORTED
from 
RAW.USECASE_TABLE limit 100000;

select * from RAW.HIGHWAY_SAFETY limit 10;
--------------------------------------------------------------------------------

The step below requires the Marketplace DB:
- US_POINTS_OF_INTEREST__ADDRESSES.CYBERSYN.US_ADDRESSES
- Get it from > Data products > Marketplace > US Addresses POI
- if you have imported the db as a different name then update lines 28 and 53 accordingly


In [ ]:
-- Leverage Snowflake Marketplace Address Data
--    Data products > Marketplace > US Addresses POI 
-- Create a table with random addresses that will be joined to the table created in the prior step
-- Update the where clause to support your story, filter by specific zip, city,  state, etc....
-- The result will be a table with GEO Coded address, each record will also have a uniqe ROW_ID
--
-- Create empty table to get the structure
--
 create or replace table RAW.POI_ADDRESSES as
    SELECT
        ADDRESS_ID,
        ID_ZIP,
        ID_CITY,
        ID_STATE,
        ID_COUNTRY,
        LONGITUDE,
        LATITUDE,
        NUMBER,
        STREET_DIRECTIONAL_PREFIX,
        STREET,
        STREET_TYPE,
        STREET_DIRECTIONAL_SUFFIX,
        UNIT,
        CITY,
        STATE,
        ZIP     
    FROM
        US_POINTS_OF_INTEREST__ADDRESSES.CYBERSYN.US_ADDRESSES
    limit 0;
--------------------------------------------------------------------------------


-- create sequence then a ROW_ID populated by the sequence, this will join to address info in FL
--
CREATE OR REPLACE SEQUENCE RAW.POI_ADDRESSES_ROW_ID_SEQ
  START WITH 1
  INCREMENT BY 1;

alter table RAW.POI_ADDRESSES add column
ROW_ID NUMBER(38,0) DEFAULT RAW.POI_ADDRESSES_ROW_ID_SEQ.NEXTVAL;
-------------------------------------------------------------------------------- 



-- add data to new table and verify the sequenc is working
-- Update the where clause to support your story, filter by specific zip, city,  state, etc....
--
insert into RAW.POI_ADDRESSES
( ADDRESS_ID, ID_ZIP, ID_CITY, ID_STATE, ID_COUNTRY, LONGITUDE, LATITUDE, NUMBER, STREET_DIRECTIONAL_PREFIX, STREET, STREET_TYPE, STREET_DIRECTIONAL_SUFFIX, UNIT, CITY, STATE, ZIP)
select 
ADDRESS_ID, ID_ZIP, ID_CITY, ID_STATE, ID_COUNTRY, LONGITUDE, LATITUDE, NUMBER, STREET_DIRECTIONAL_PREFIX, STREET, STREET_TYPE, STREET_DIRECTIONAL_SUFFIX, UNIT, CITY, STATE, ZIP
from 
US_POINTS_OF_INTEREST__ADDRESSES.CYBERSYN.US_ADDRESSES
WHERE
        STATE = 'FL' 
      -- and  ZIP in ('32301', '32202', '32801' )
limit 10000;

select * from RAW.POI_ADDRESSES limit 10;
--------------------------------------------------------------------------------


    
     

In [ ]:
-- check to see max and min values, it's not a perfect sequence, but will suffice for demo purposes
--
select max(row_id), min(row_id) from RAW.POI_ADDRESSES;

In [ ]:
-- See how many rows will be produced when Safety and Address data is joined
-- Should be roughly the lower of limit on two above create tables
-- Sequence is not a perfect incremental counter
-- Should suffice for demo purposes
--
SELECT COUNT(*)  FROM 
RAW.POI_ADDRESSES FL,
RAW.HIGHWAY_SAFETY A
WHERE
FL.ROW_ID=A.ROW_ID

Real world, one can join by address fields or employ other Geo Coding techniques, some relevant resources:
- Quick Start - [Using Snowflake Cortex And Streamlit With Geospatial Data](https://quickstarts.snowflake.com/guide/using_snowflake_cortex_and_streamlit_with_geospatial_data/index.html?index=..%2F..index#0)
- Medium Article - [How to Geocode Addresses in Snowflake](https://medium.com/snowflake/how-to-geocode-addresses-in-snowflake-ea1b6ccdde24)


- Medium Article - [Geocode Addrsses in Snowflake with External Access Integration](  https://medium.com/snowflake/geocode-addresses-in-snowflake-with-external-access-integration-4237103659ef)

In [ ]:
-- Check for distinct ROW_ID
-- Query should return empty or 0, only want to use each address once
--
SELECT DISTINCT A.ROW_ID FROM 
RAW.POI_ADDRESSES FL,
RAW.HIGHWAY_SAFETY A
WHERE
FL.ROW_ID=A.ROW_ID
GROUP BY A.ROW_ID
HAVING COUNT(A.ROW_ID) > 1

In [ ]:
SELECT
    A.* EXCLUDE(ROW_ID), FL.*   
FROM 
    RAW.POI_ADDRESSES FL,
    RAW.HIGHWAY_SAFETY A
WHERE
    FL.ROW_ID=A.ROW_ID limit 10;

In [ ]:
-- This could also be done via a view or perhaps leverage dynamic tables
--
create or replace table 
    SILVER.HIGHWAY_SAFETY_GEO as
SELECT
    A.* EXCLUDE(ROW_ID), FL.*   
FROM 
    RAW.POI_ADDRESSES FL,
    RAW.HIGHWAY_SAFETY A
WHERE
    FL.ROW_ID=A.ROW_ID 

In [ ]:
-- My demo story is to find expired vehicle registrations
--
select min(REGISTRATION_EXPIRATION), max(REGISTRATION_EXPIRATION)  from SILVER.HIGHWAY_SAFETY_GEO;
select count(*) from SILVER.HIGHWAY_SAFETY_GEO where REGISTRATION_EXPIRATION < CURRENT_DATE;

In [ ]:
# Let's see a H3 map of our data, we chose a level of 5, update as appropriate or make it dynamic
# Hover over a cell and see the number of expired registrations
#
import streamlit as st
import pandas as pd
import pydeck as pdk
import numpy as np

# Query to get map points (H3 index) and aggregate COUNT for each H3 cell
df_query = """
SELECT 
    H3_LATLNG_TO_CELL_STRING(LATITUDE, LONGITUDE, 5) AS LATLONG_5, 
    COUNT(*) AS COUNT , AVG(LATITUDE) as AVG_LATITUDE, AVG(LONGITUDE) as AVG_LONGITUDE
FROM SILVER.HIGHWAY_SAFETY_GEO 
WHERE REGISTRATION_EXPIRATION < CURRENT_DATE
GROUP BY LATLONG_5
LIMIT 5000;
"""
df = session.sql(df_query).to_pandas()

# Check the data to make sure LATLONG_9 is in the expected format
# st.write(df.head())  # Inspect first few rows of your DataFrame

# Define a function to calculate RGB color based on COUNT
def get_color_for_count(count):
    # Normalize the COUNT value to be between 0 and 255
    count = max(0, min(count, 255))  # Ensure COUNT is between 0 and 255
    
    # Color logic: As COUNT increases, shift from blue (low) to red (high)
    red = 255 - count  # Red decreases as COUNT increases
    green = 255
    blue = count  # Blue increases as COUNT increases
    
    return [red, green, blue]

# Apply the color function to the COUNT column in your DataFrame
df['color'] = df['COUNT'].apply(get_color_for_count)


# Define tooltip, to provide details on points
tooltip = {
        "html": 
                "<b>Number of Records:</b> {COUNT} <br/>"
              "<b>RGB Color:</b> {color} <br/>",
                 
         
            "style": {
                    "backgroundColor": 'rgba(128, 0, 128, 0.5)',
                    "color": "white"}}



# Calcualte average to set centerpoint of map, leverages above dataframe
avg_latitude =  df['AVG_LATITUDE'].mean()
avg_longitude =  df['AVG_LONGITUDE'].mean()




# Create the pydeck Layer with tooltip
layer = pdk.Layer(
    "H3HexagonLayer",
    df,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=False,
    opacity=0.1,
    get_hexagon="LATLONG_5",  # Use the H3 index column to determine hexagon position
    #get_fill_color="color",   # Use the dynamically calculated color
    get_fill_color="color",   # Use the dynamically calculated color
    #get_line_color=[255, 255, 255],  # White line color
    get_line_color=[0,0,255],  # White line color
    line_width_min_pixels=2,

)

# Mapbox API key (ensure you have a valid key here)
mbkey = ''  # Replace with your actual Mapbox API key

# Define the map details
deck_all_layers = pdk.Deck(
    map_provider='mapbox',
    api_keys={"mapbox": mbkey},
    map_style='mapbox://styles/mapbox/light-v11',
    layers=[layer],
    tooltip=tooltip,
    initial_view_state=pdk.ViewState(
       # latitude=28.538336,  # Adjust if necessary to center on your data
       # longitude=-81.379234,  # Adjust if necessary to center on your data

latitude=avg_latitude ,  # Adjust if necessary to center on your data
        longitude=avg_longitude,  # Adjust if necessary to center on your data

        
        zoom=7,  # Try adjusting the zoom level
        pitch=30
    )
)




 


st.write("DHS & MV - Expired Registrations")

# Display the map
st.pydeck_chart(deck_all_layers)


In [ ]:
# Let's view the points on a map
#
import streamlit as st
import pandas as pd
import pydeck as pdk
 
 
# Query to get my map points
map_locations_query = f"""select LONGITUDE, LATITUDE , VIN , ZIP, cast(REGISTRATION_EXPIRATION as string) as REGISTRATION_EXPIRATION_STR from SILVER.HIGHWAY_SAFETY_GEO where REGISTRATION_EXPIRATION < CURRENT_DATE"""
map_locations_query_df = session.sql(map_locations_query).to_pandas()
map_locations_query_col = session.sql(map_locations_query).collect()



# Define my map layer off above dataframe
map_locations_layer = pdk.Layer(
    type='ScatterplotLayer', 
    data=map_locations_query_df, 
    pickable=True,
    get_position='[LONGITUDE, LATITUDE]', 
    get_color='[215,21,99]',
    # get_color='[128,0,128,200]',   
    get_radius=300,
    opacity=0.1,
    auto_highlight=True,
    id = 'map_locations_id' 
)


# Calcualte average to set centerpoint of map, leverages above dataframe
avg_latitude = map_locations_query_df['LATITUDE'].mean()
avg_longitude = map_locations_query_df['LONGITUDE'].mean()


# Define tooltip, to provide details on points
tooltip = {
        "html": 
                "<b>VIN:</b> {VIN} <br/>"
                "<b>Registration Expiration:</b> {REGISTRATION_EXPIRATION_STR} <br/>"
                "<b>ZIP:</b> {ZIP}",
            "style": {
                    "backgroundColor": 'rgba(128, 0, 128, 0.5)',
                    "color": "white"}}

# mapbox API key
mbkey = ''

# define map details - layers, map provider, initial state, tooltip
deck_all_layers = pdk.Deck(
        map_provider = 'mapbox',
        api_keys = {"mapbox":mbkey},
        map_style = 'mapbox://styles/mapbox/light-v11',
        layers=[
             map_locations_layer
               ], 
        tooltip = tooltip,
        initial_view_state=pdk.ViewState(
        latitude=avg_latitude,
        longitude=avg_longitude,
        zoom=7,
        pitch=30
        )
        )
st.write("DHS & MV - Expired Registrations")
# display the map
st.pydeck_chart(deck_all_layers)


In [ ]:
-- Optional Step
--    to show how more compute can be added on the demand 
--    update warehouse name based on your environment
alter warehouse XS_WAREHOUSE  set warehouse_size = 'Large';

In [ ]:
-- Find registration address within X miles of a Field Office
-- Display the true false field and the distance
--
----------------------------------------------
-- SET VARIABLES 
set MILES=10;
set FIELD_OFFICE_LAT=30.21932;
set FIELD_OFFICE_LONG=-81.594196;
----------------------------------------------

-- there are 1609.34 meters in a mile
-- CTE to allow us to output only the fields we want
with POINTS as (
SELECT 
  ST_MAKEPOINT(LONGITUDE, LATITUDE) as VEHICLE_LOCATION_POINT,
  ST_MAKEPOINT(($FIELD_OFFICE_LONG),($FIELD_OFFICE_LAT)) as FIELD_OFFICE_POINT , 
  VIN,
  NUMBER || ' ' || STREET || ' ' || CITY || ', ' || STATE || ' ' || ZIP as ADDRESS,
  LATITUDE,
  LONGITUDE,
  ZIP,
  cast(REGISTRATION_EXPIRATION as string) as REGISTRATION_EXPIRATION_STR
FROM 
     SILVER.HIGHWAY_SAFETY_GEO
     )
 
-- Select against CTE
SELECT 
  VIN,
  ADDRESS,  
  ST_DWITHIN(VEHICLE_LOCATION_POINT, FIELD_OFFICE_POINT , (($MILES)*1609.34)) AS WITHIN_RANGE,
  ST_DISTANCE(VEHICLE_LOCATION_POINT, FIELD_OFFICE_POINT )/1609.34 AS MILES_AWAY,
  LATITUDE,
  LONGITUDE,
  ZIP,
  REGISTRATION_EXPIRATION_STR
FROM 
  POINTS
WHERE
  ST_DISTANCE(VEHICLE_LOCATION_POINT, FIELD_OFFICE_POINT ) < (($MILES)*1609.34) and 
  VIN is not NULL;


In [ ]:
-- Resize warehouse back to X-Small, if you ran the cell to change to Large 
--    update warehouse name based on your environment
alter warehouse XS_WAREHOUSE  set warehouse_size = 'X-Small';

In [ ]:
# Let's create an interactive map that allows users to select a field office and 
#   see expired registrations within x miles

import streamlit as st
import pandas as pd
import pydeck as pdk
 
st.write("")
st.write("DHS & MV - Expired Registrations")
st.write("")

# -------------------------------------------------------------------------------------
# Filter for city center
# -------------------------------------------------------------------------------------

# Sidebar filter for the number of stations
#MILES_AWAY = st.sidebar.number_input("Distance away (miles):", min_value=1, max_value=10, step=1, value=1)

col2, col1 = st.columns(2)

with col1:
    MILES_AWAY_NUM = st.number_input("Distance away (miles):", min_value=1, max_value=10, step=1, value=1)


city_coordinates = {
    "Jacksonville": (-81.6557,30.3322),
    "Orlando": (-81.3789,28.5384),
    "Tallahassee": (-84.2806,30.4382)
}

with col2:
    selected_city = st.selectbox(
    "Select a city:",
    options=list(city_coordinates.keys())  # The dropdown options (city names)
)


# Get the lat/long from the dictionary based on the selected city
selected_lon, selected_lat = city_coordinates[selected_city]



 
map_locations_query = f"""
with POINTS as (
SELECT 
  ST_MAKEPOINT(LONGITUDE, LATITUDE) as VEHICLE_LOCATION_POINT,
  ST_MAKEPOINT({selected_lon},{selected_lat}) as FIELD_OFFICE_POINT , 
  VIN,
  NUMBER || ' ' || STREET || ' ' || CITY || ', ' || STATE || ' ' || ZIP as ADDRESS,
  LATITUDE,
  LONGITUDE,
  ZIP,
  cast(REGISTRATION_EXPIRATION as string) as REGISTRATION_EXPIRATION_STR
FROM 
     SILVER.HIGHWAY_SAFETY_GEO
     )
 
-- Select against CTE
SELECT 
  VIN,
  ADDRESS,  
  ST_DWITHIN(VEHICLE_LOCATION_POINT, FIELD_OFFICE_POINT , ({MILES_AWAY_NUM}*1609.34)) AS WITHIN_RANGE,
  ST_DISTANCE(VEHICLE_LOCATION_POINT, FIELD_OFFICE_POINT )/1609.34 AS MILES_AWAY,
  LATITUDE,
  LONGITUDE,
  ZIP,
  REGISTRATION_EXPIRATION_STR
FROM 
  POINTS
WHERE
  ST_DISTANCE(VEHICLE_LOCATION_POINT, FIELD_OFFICE_POINT ) < ({MILES_AWAY_NUM}*1609.34) and 
  VIN is not NULL;
 """



# Query to get my map points
#map_locations_query = f"""select LONGITUDE, LATITUDE , VIN , ZIP, cast(REGISTRATION_EXPIRATION as string) as REGISTRATION_EXPIRATION_STR from FL_DHS_MV.SILVER.HIGHWAY_SAFETY_GEO where REGISTRATION_EXPIRATION < CURRENT_DATE"""
map_locations_query_df = session.sql(map_locations_query).to_pandas()
map_locations_query_col = session.sql(map_locations_query).collect()



# Define my map layer off above dataframe
map_locations_layer = pdk.Layer(
    type='ScatterplotLayer', 
    data=map_locations_query_df, 
    pickable=True,
    get_position='[LONGITUDE, LATITUDE]', 
    get_color='[215,21,99]',
    # get_color='[128,0,128,200]',   
    get_radius=100,
    opacity=0.1,
    auto_highlight=True,
    id = 'map_locations_id' 
)


# Calcualte average to set centerpoint of map, leverages above dataframe
avg_latitude = map_locations_query_df['LATITUDE'].mean()
avg_longitude = map_locations_query_df['LONGITUDE'].mean()


# Define tooltip, to provide details on points
tooltip = {
        "html": 
                "<b>VIN:</b> {VIN} <br/>"
                "<b>Registration Expiration:</b> {REGISTRATION_EXPIRATION_STR} <br/>"
                "<b>ZIP:</b> {ZIP}",
            "style": {
                    "backgroundColor": 'rgba(128, 0, 128, 0.5)',
                    "color": "white"}}




# mapbox API key
mbkey = ''

# define map details - layers, map provider, initial state, tooltip
deck_all_layers = pdk.Deck(
        map_provider = 'mapbox',
        api_keys = {"mapbox":mbkey},
        map_style = 'mapbox://styles/mapbox/light-v11',
        layers=[
             map_locations_layer
               ], 
        tooltip = tooltip,
        initial_view_state=pdk.ViewState(
        latitude=avg_latitude,
        longitude=avg_longitude,
        zoom=11,
        pitch=30
        )
        )

# display the map
st.pydeck_chart(deck_all_layers)

In [ ]:
# Let's create an interactive map that allows users to select a field office and 
#   see expired registrations within x miles

import streamlit as st
import pandas as pd
import pydeck as pdk
 


st.write("")
st.write("DHS & MV - Expired Registrations - Four Random Points")
st.write("")

# -------------------------------------------------------------------------------------
# Filter for city center
# -------------------------------------------------------------------------------------

# Sidebar filter for the number of stations
#MILES_AWAY = st.sidebar.number_input("Distance away (miles):", min_value=1, max_value=10, step=1, value=1)



city_coordinates = {
    "Jacksonville": (-81.6557,30.3322),
    "Orlando": (-81.3789,28.5384),
    "Tallahassee": (-84.2806,30.4382)
}

 
selected_city = st.selectbox(
    "Select a city:",
    options=list(city_coordinates.keys())  # The dropdown options (city names)
)


# Get the lat/long from the dictionary based on the selected city
selected_lon, selected_lat = city_coordinates[selected_city]

 
map_locations_query = f"""
with POINTS as (
SELECT 
  ST_MAKEPOINT(LONGITUDE, LATITUDE) as VEHICLE_LOCATION_POINT,
  ST_MAKEPOINT({selected_lon},{selected_lat}) as FIELD_OFFICE_POINT , 
  VIN,
  NUMBER || ' ' || STREET || ' ' || CITY || ', ' || STATE || ' ' || ZIP as ADDRESS,
  LATITUDE,
  LONGITUDE,
  ZIP,
  cast(REGISTRATION_EXPIRATION as string) as REGISTRATION_EXPIRATION_STR
FROM 
     SILVER.HIGHWAY_SAFETY_GEO
     )
 
-- Select against CTE
SELECT 
  VIN,
  ADDRESS,  
  ST_DWITHIN(VEHICLE_LOCATION_POINT, FIELD_OFFICE_POINT , ({MILES_AWAY_NUM}*1609.34)) AS WITHIN_RANGE,
  ST_DISTANCE(VEHICLE_LOCATION_POINT, FIELD_OFFICE_POINT )/1609.34 AS MILES_AWAY,
  LATITUDE,
  LONGITUDE,
  ZIP,
  REGISTRATION_EXPIRATION_STR
FROM 
  POINTS
WHERE
  ST_DISTANCE(VEHICLE_LOCATION_POINT, FIELD_OFFICE_POINT ) < ({MILES_AWAY_NUM}*1609.34) and 
  VIN is not NULL limit 4;
 """



# Query to get my map points
#map_locations_query = f"""select LONGITUDE, LATITUDE , VIN , ZIP, cast(REGISTRATION_EXPIRATION as string) as REGISTRATION_EXPIRATION_STR from FL_DHS_MV.SILVER.HIGHWAY_SAFETY_GEO where REGISTRATION_EXPIRATION < CURRENT_DATE"""
map_locations_query_df = session.sql(map_locations_query).to_pandas()
map_locations_query_col = session.sql(map_locations_query).collect()


# Ensure to use `.loc` for adding new columns to avoid SettingWithCopyWarning
map_locations_query_df_arc = map_locations_query_df[['LONGITUDE', 'LATITUDE', 'ZIP', 'VIN', 'REGISTRATION_EXPIRATION_STR']].copy()

# Now you can safely add the new columns without getting the warning
map_locations_query_df_arc.loc[:, 'selected_lon'] = selected_lon
map_locations_query_df_arc.loc[:, 'selected_lat'] = selected_lat


# RGBA. A = Alpha, it represents transparency.
#    0 = fully transparent, 1 = fully opaque, defaults to 100
GREEN_RGB = [0, 255, 0, 90]
RED_RGB = [240, 100, 0, 90]



# Define the ArcLayer
arc_layer = pdk.Layer(
    'ArcLayer',
    data=map_locations_query_df_arc,
    get_source_position=[selected_lon, selected_lat],  # Source is the selected city coordinates
    get_target_position=['LONGITUDE', 'LATITUDE'],  # Target is the latitude/longitude from the DataFrame
    #get_color='color',
    #get_width='width',
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=GREEN_RGB,
    getWidth=4,
    getHeight=2,
    getOpacity=1,
    pickable=True,
    auto_highlight=True

)

# st.write(arc_layer)

# Define the Deck object and display it
deck = pdk.Deck(layers=[arc_layer], 
        map_provider = 'mapbox',
        api_keys = {"mapbox":mbkey},
        map_style = 'mapbox://styles/mapbox/light-v11',
        initial_view_state=pdk.ViewState(
            latitude=37.7749, 
            longitude=-122.4194, 
            zoom=3)
               )


# Calcualte average to set centerpoint of map, leverages above dataframe
avg_latitude = map_locations_query_df['LATITUDE'].mean()
avg_longitude = map_locations_query_df['LONGITUDE'].mean()


# Define tooltip, to provide details on points
tooltip = {
        "html": 
                "<b>VIN:</b> {VIN} <br/>"
                "<b>Registration Expiration:</b> {REGISTRATION_EXPIRATION_STR} <br/>"
                "<b>ZIP:</b> {ZIP}",
            "style": {
                    "backgroundColor": 'rgba(128, 0, 128, 0.5)',
                    "color": "white"}}




# mapbox API key
mbkey = ''

# define map details - layers, map provider, initial state, tooltip
deck_all_layers = pdk.Deck(
        map_provider = 'mapbox',
        api_keys = {"mapbox":mbkey},
        map_style = 'mapbox://styles/mapbox/light-v11',
        layers=[
            arc_layer
               ], 
        tooltip = tooltip,
        initial_view_state=pdk.ViewState(
        latitude=avg_latitude,
        longitude=avg_longitude,
        zoom=13,
        pitch=30
        )
        )

# display the map
st.pydeck_chart(deck_all_layers)

In [ ]:

# Let's create an interactive map that allows users to select a field office and 
#   see expired registrations within x miles

import streamlit as st
import pandas as pd
import pydeck as pdk
from pydeck.types import String
 

st.write("")
st.write("DHS & MV - Expired Registrations - Random Points - Path & Text Layer")
st.write("")

# -------------------------------------------------------------------------------------
# Filter for city center
# -------------------------------------------------------------------------------------

# Sidebar filter for the number of stations
#MILES_AWAY = st.sidebar.number_input("Distance away (miles):", min_value=1, max_value=10, step=1, value=1)


city_coordinates = {
    "Jacksonville": (-81.6557,30.3322),
    "Orlando": (-81.3789,28.5384),
    "Tallahassee": (-84.2806,30.4382)
}


selected_city = st.selectbox(
    "Select a city:",
    options=list(city_coordinates.keys())  # The dropdown options (city names)
)


# Get the lat/long from the dictionary based on the selected city
selected_lon, selected_lat = city_coordinates[selected_city]



 
map_locations_query = f"""
with POINTS as (
SELECT 
  ST_MAKEPOINT(LONGITUDE, LATITUDE) as VEHICLE_LOCATION_POINT,
  ST_MAKEPOINT({selected_lon},{selected_lat}) as FIELD_OFFICE_POINT , 
  VIN,
  NUMBER || ' ' || STREET || ' ' || CITY || ', ' || STATE || ' ' || ZIP as ADDRESS,
  LATITUDE,
  LONGITUDE,
  ZIP,
  cast(REGISTRATION_EXPIRATION as string) as REGISTRATION_EXPIRATION_STR
FROM 
     SILVER.HIGHWAY_SAFETY_GEO
     )
 
-- Select against CTE
SELECT 
  VIN,
  ADDRESS,  
  ST_DWITHIN(VEHICLE_LOCATION_POINT, FIELD_OFFICE_POINT , ({MILES_AWAY_NUM}*1609.34)) AS WITHIN_RANGE,
  ST_DISTANCE(VEHICLE_LOCATION_POINT, FIELD_OFFICE_POINT )/1609.34 AS MILES_AWAY,
  LATITUDE,
  LONGITUDE,
  ZIP,
  REGISTRATION_EXPIRATION_STR
FROM 
  POINTS
WHERE
  ST_DISTANCE(VEHICLE_LOCATION_POINT, FIELD_OFFICE_POINT ) < ({MILES_AWAY_NUM}*1609.34) and 
  VIN is not NULL limit 4;
 """



# Query to get my map points
#map_locations_query = f"""select LONGITUDE, LATITUDE , VIN , ZIP, cast(REGISTRATION_EXPIRATION as string) as REGISTRATION_EXPIRATION_STR from FL_DHS_MV.SILVER.HIGHWAY_SAFETY_GEO where REGISTRATION_EXPIRATION < CURRENT_DATE"""
map_locations_query_df = session.sql(map_locations_query).to_pandas()
map_locations_query_col = session.sql(map_locations_query).collect()



def prepare_path_data(map_locations_query_df):
        return map_locations_query_df[['LONGITUDE', 'LATITUDE']].values.tolist()

path_data = prepare_path_data(map_locations_query_df)




path_layer = pdk.Layer(
    type="PathLayer",
    data=[{'coordinates': path_data}],  # Wrap path_data in a dict with a 'coordinates' key
    get_path="coordinates",  # Ensure it refers to the 'coordinates' field in the data
    get_color=[255, 0, 0],  # Red color for the path
    width_scale=20,
    width_min_pixels=2,
)



# Define my map layer off above dataframe
map_locations_layer = pdk.Layer(
    type='ScatterplotLayer', 
    data=map_locations_query_df, 
    pickable=True,
    get_position='[LONGITUDE, LATITUDE]', 
    get_color='[215,21,99]',
    # get_color='[128,0,128,200]',   
    get_radius=10,
    opacity=0.1,
    auto_highlight=True,
    id = 'map_locations_id' 
)



st.write(map_locations_query_df)


# Calcualte average to set centerpoint of map, leverages above dataframe
avg_latitude = map_locations_query_df['LATITUDE'].mean()
avg_longitude = map_locations_query_df['LONGITUDE'].mean()


# Define tooltip, to provide details on points
tooltip = {
        "html": 
                "<b>VIN:</b> {VIN} <br/>"
                "<b>Registration Expiration:</b> {REGISTRATION_EXPIRATION_STR} <br/>"
                "<b>ZIP:</b> {ZIP}",
            "style": {
                    "backgroundColor": 'rgba(128, 0, 128, 0.5)',
                    "color": "white"}}



text_layer = pdk.Layer(
    "TextLayer",
    data=map_locations_query_df,
    pickable=True,
    get_position='[LONGITUDE, LATITUDE]',
    get_text="VIN",
    get_size=16,
    get_color=[0, 0, 0],
    get_angle=0,
    # Note that string constants in pydeck are explicitly passed as strings
    # This distinguishes them from columns in a data set
    get_text_anchor=String("middle"),
    get_alignment_baseline=String("center"),
)






# mapbox API key
mbkey = ''

# define map details - layers, map provider, initial state, tooltip
deck_all_layers = pdk.Deck(
        map_provider = 'mapbox',
        api_keys = {"mapbox":mbkey},
        map_style = 'mapbox://styles/mapbox/light-v11',
        layers=[
             map_locations_layer,
            path_layer,
            text_layer
               ], 
        tooltip = tooltip,
        initial_view_state=pdk.ViewState(
        latitude=avg_latitude,
        longitude=avg_longitude,
        zoom=11,
        pitch=30
        )
        )

# display the map
st.pydeck_chart(deck_all_layers)